# Global Minimial Viable Products (1)
This is only for CONUS currently as the potential runoff product is to large to be an asset. 

Combine:
* SMAP available porosity
* ERA5-Merit potential runoff
* MERIT terrain slope
* MERIT upstream contributing area (needs to be less than some threshold)
* (ERA5 Moisture values)

In to various "products".

In [1]:
import ee
import numpy as np
from statistics import mean
import folium
from folium import plugins

# !pip install earthshot
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
# Some common parameters
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_per_month = [
    31, 28, 31, 30, 31, 30,
    31, 31, 30, 31, 30, 31] 
seconds_per_month = [ dd* 24*60*60 for dd in days_per_month ]
month = 4
month_name = month_names[month - 1]

bbox = common.bboxes()['conus']

# SMAP-USDA Available Porosity
More available porosity is better.

In [4]:
# Get a single example image 
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')

avail_porosity_img = (
    smap_usda_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('band', 'avail_porosity_mm')).first())

avail_porosity_img_scaled = norm.img_scale(
    avail_porosity_img, area_of_interest=common.bboxes()['conus'])

In [5]:
smap_missing = -99999

# units: mm
avail_porosity_data = np.array(
        avail_porosity_img
        .sampleRectangle(bbox, ['avail_porosity_mm'], smap_missing)
        .get('avail_porosity_mm')
        .getInfo())
avail_porosity_data_mask = np.ma.masked_values(avail_porosity_data, smap_missing)
avail_porosity_data_drop = avail_porosity_data_mask[~avail_porosity_data_mask.mask]

# Scaled
avail_porosity_scaled_data = np.array(
        avail_porosity_img_scaled
        .sampleRectangle(bbox, ['avail_porosity_mm'], smap_missing)
        .get('avail_porosity_mm')
        .getInfo())
avail_porosity_scaled_data_mask = np.ma.masked_values(avail_porosity_scaled_data, smap_missing)
avail_porosity_scaled_data_drop = avail_porosity_scaled_data_mask[~avail_porosity_scaled_data_mask.mask]

## ERA5-MERIT Potential Runoff

In [6]:
# Get a single example image 
pot_runoff_clim = ee.ImageCollection(
    'users/jamesmcc/era5_merit_potential_sfc_runoff_conus_climatology')

pot_runoff_img = (
    pot_runoff_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('variable', 'potential_sfc_runoff_mon_clim_cms')).first())

# Mask to a usable range
runoff_mask = pot_runoff_img.lte(.5).And(pot_runoff_img.gte(.01))

pot_runoff_img_scaled = norm.img_scale(
    pot_runoff_img.updateMask(runoff_mask), 
    area_of_interest=common.bboxes()['conus'])

## ERA5 Soil Moisture Climatology

In [7]:
# Do the same for ERA5 moisture, get a single example image 
pot_moisture_clim = ee.ImageCollection(
    'users/amgadellaboudy/era5_vol_soil_moisture_conus_climatology1')

pot_moisture_img = (
    pot_moisture_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('variable', 'potential_sfc_moisture_mon_clim_cms')).first())

# No masking for moisture values
#moisture_mask = pot_moisture_img.lte(.5).And(pot_moisture_img.gte(.01))

pot_moisture_img_scaled = norm.img_scale(
    pot_moisture_img, 
    area_of_interest=common.bboxes()['conus'])

## MERIT Terrain Slope

In [8]:
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(9).And(slope_img.gte(2))

In [9]:
# This dosent really need scaled, just a mask?
#slope_img_scaled = norm.img_scale(slope_img, area_of_interest=common.bboxes()['conus'])

## Region Of Interest

In [10]:
coos_county_parcels = ee.FeatureCollection(
    'users/tsongkapa/Coos_county_parcels')
douglas_county_parcels = ee.FeatureCollection(
    'users/tsongkapa/Douglas_county_parcels')

In [11]:
i= coos_county_parcels.getMapId()
print(i)

{'mapid': 'projects/earthengine-legacy/maps/60ee0ab889747aabefa1de3efd3350ad-f35089235a48e34ab0361bff7fe36930', 'token': '', 'tile_fetcher': <ee.data.TileFetcher object at 0x7fd229f3b8e0>, 'image': <ee.image.Image object at 0x7fd229f3b6d0>}


## Product 1
Simply give higher scores to places that have both high potential runoff and available porosity (and with low moisture), in only areas with acceptable slopes

In [12]:
product_1 = (
    avail_porosity_img_scaled
    .multiply(pot_runoff_img_scaled)
    .divide(pot_moisture_img_scaled)
    .updateMask(slope_mask)
    .updateMask(runoff_mask))

In [13]:
#understanding product_1 range

product_1_range = norm.img_range(product_1, area_of_interest = bbox)

palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=product_1_range[0], maximum= product_1_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': product_1_range[0], 'max': product_1_range[1], 'dimensions': 512,
    'palette': palette}



In [14]:
#sum up product_1 scores over douglas and coos county parcels to obtain a parcel score, obtain image, obtain image for bounding box we're interested in
parcel_score_douglas = product_1.reduceRegions(collection= douglas_county_parcels, reducer= ee.Reducer.sum(), scale = 30)
parcel_score_douglas_2 = parcel_score_douglas.reduceToImage(properties = ['sum'], reducer = ee.Reducer.max())
parcel_score_coos = product_1.reduceRegions(collection= coos_county_parcels, reducer= ee.Reducer.sum(), scale = 30)
parcel_score_coos_2 = parcel_score_coos.reduceToImage(properties = ['sum'], reducer = ee.Reducer.max())
bound_box = ee.Feature(ee.Geometry.Polygon([[[-124.1, 43.38], [-124.1, 42.88], [-123.6, 42.88], [-123.6, 43.38]]]))
bound_box_img = bound_box.getMapId()['image']

bound_box_or = ee.Feature(ee.Geometry.BBox(-124.566244,41.991794, -116.463504, 46.292035))
bound_box_or_4real = ee.Geometry.BBox(-124.566244,41.991794, -116.463504, 46.292035)
bound_box_or_img = bound_box_or.getMapId()['image']

In [15]:
#creating the legend for parcel scores

palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=0, maximum= 10)


vis_params_2 = {
    'min': 0, 'max': 10,'dimensions': 512,
    'palette': palette}



In [16]:
#creating the map, visualizing parcel score data for parcels in douglas and coos counties

the_map = vis.folium_map(location=[43.25, -123.5], zoom_start=8, height=500)
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_name = month_names[month-1]

#douglas_img = douglas_county_parcels.draw(color = 'green', strokeWidth= 1)
#coos_img = coos_county_parcels.draw(color = 'green', strokeWidth= 1)

the_map.add_ee_layer(parcel_score_douglas_2, vis_params_2, name = 'Parcel Score_douglas')
the_map.add_ee_layer(parcel_score_coos_2, vis_params_2, name = 'Parcel Score_coos')
the_map.add_ee_layer(bound_box_img, {}, name = 'Bounding_Box')
#the_map.add_ee_layer(bound_box_or_img, {}, name = 'Bounding_Box_OR')

vis.folium_display(the_map)


## Export

In [17]:
collection_name = 'users/jamesmcc/mvp_moisture_amgad'
runoff_asset = ee.data.createAsset(
    {'type': 'ImageCollection'}, collection_name)

img_dict = {
    'parcel_score_coos_co': parcel_score_coos_2, 
    'parcel_score_douglas_co': parcel_score_douglas_2}
for desc, img in img_dict.items():
    xx = ee.batch.Export.image.toAsset(
                ee.Image(img),
                description=desc, 
                assetId=runoff_asset['id'] + '/' + desc,
                region=bound_box_or_4real, 
                #maxPixels=max_pixels
    ).start()